Data insert and cleaning 

In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Date, Boolean, Float, DateTime, func
import urllib

#connection string
params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=AZTRIP2\\SQLEXPRESS;"  
    "DATABASE=OLP_Engagement_Analytics;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")
metadata = MetaData()

# 1. Users
users = Table('Users', metadata,
    Column('user_id', Integer, primary_key=True),
    Column('name', String(255)),
    Column('email', String(255)),
    Column('role', String(50)),
    Column('birth_date', Date),
    Column('gender', String(10)),
    Column('country', String(100)),
    Column('device', String(50)),
    Column('subscription_type', String(50)),
    Column('registration_date', Date),
    Column('last_login_date', Date)
)

# 2. Courses
courses = Table('Courses', metadata,
    Column('course_id', Integer, primary_key=True),
    Column('course_name', String(255)),
    Column('instructor_name', String(255)),
    Column('category', String(100)),
    Column('difficulty_level', String(50)),
    Column('language', String(50)),
    Column('created_date', Date),
    Column('duration_hours', Integer)
)

# 3. Enrollments
enrollments = Table('Enrollments', metadata,
    Column('enrollment_id', Integer, primary_key=True),
    Column('user_id', Integer),
    Column('course_id', Integer),
    Column('enrollment_date', Date),
    Column('completed', Boolean),
    Column('completion_date', Date),
    Column('final_grade', Float),
    Column('certificate_earned', Boolean)
)

# 4. Activity_Logs
logs = Table('Activity_Logs', metadata,
    Column('log_id', Integer, primary_key=True),
    Column('user_id', Integer),
    Column('course_id', Integer),
    Column('login_datetime', DateTime),
    Column('session_duration_min', Integer),
    Column('pages_viewed', Integer),
    Column('video_watched_min', Integer),
    Column('time_of_day', String(20))
)

# 5. Forum_Posts
posts = Table('Forum_Posts', metadata,
    Column('post_id', Integer, primary_key=True),
    Column('user_id', Integer),
    Column('course_id', Integer),
    Column('post_datetime', DateTime),
    Column('content_length', Integer),
    Column('likes_received', Integer),
    Column('replies_received', Integer)
)

# 6. Quizzes
quizzes = Table('Quizzes', metadata,
    Column('quiz_id', Integer, primary_key=True),
    Column('course_id', Integer),
    Column('quiz_title', String(255)),
    Column('total_questions', Integer),
    Column('pass_score', Integer),
    Column('created_date', Date)
)

# 7. Quiz_Attempts
attempts = Table('Quiz_Attempts', metadata,
    Column('attempt_id', Integer, primary_key=True),
    Column('user_id', Integer),
    Column('quiz_id', Integer),
    Column('attempt_date', Date),
    Column('score', Integer),
    Column('passed', Boolean)
)

# 8. Feedback
feedback = Table('Feedback', metadata,
    Column('feedback_id', Integer, primary_key=True),
    Column('user_id', Integer),
    Column('course_id', Integer),
    Column('rating', Integer),
    Column('comments', String(1000)),
    Column('submitted_date', Date)
)

# Create all tables in the database
metadata.create_all(engine)
print("✅ Bütün cədvəllər uğurla yaradıldı.")


✅ Bütün cədvəllər uğurla yaradıldı.


In [ ]:
import pandas as pd
import pyodbc


conn = pyodbc.connect(
    r"DRIVER={ODBC Driver 17 for SQL Server};"
    r"SERVER=AZTRIP2\SQLEXPRESS;"
    r"DATABASE=OLP_Engagement_Analytics;"
    r"Trusted_Connection=yes;"
)
cursor = conn.cursor()


csv_files = {
    "Users": "users_2000.csv",
    "Courses": "courses_2000.csv",
    "Enrollments": "enrollments_2000.csv",
    "Activity_Logs": "activity_logs_2000.csv",
    "Forum_Posts": "forum_posts_2000.csv",
    "Quizzes": "quizzes_2000.csv",
    "Quiz_Attempts": "quiz_attempts_2000.csv",
    "Feedback": "feedback_2000.csv"
}


table_order = [
    "Users", "Courses", "Quizzes", "Enrollments",
    "Activity_Logs", "Forum_Posts", "Quiz_Attempts", "Feedback"
]

for table in table_order:
    df = pd.read_csv(csv_files[table])
    print(f"\n📥 {table} insert olunur...")

    cursor.execute(f"SET IDENTITY_INSERT {table} ON")

    for index, row in df.iterrows():
        columns = ', '.join(df.columns)
        placeholders = ', '.join(['?'] * len(df.columns))
        values = [row[col] for col in df.columns]

        query = f"INSERT INTO {table} ({columns}) VALUES ({placeholders})"
        try:
            cursor.execute(query, *values)
        except Exception as e:
            print(f"❌ {table} -> Sətir {index} insert Xətası: {e}")

    cursor.execute(f"SET IDENTITY_INSERT {table} OFF")
    conn.commit()
    print(f" {table} cədvəlinə insert tamamlandı.")

conn.close()
print("\n Bütün cədvəllərə data insert edildi.")







📥 Users insert olunur...
✅ Users cədvəlinə insert tamamlandı.

📥 Courses insert olunur...
✅ Courses cədvəlinə insert tamamlandı.

📥 Quizzes insert olunur...
✅ Quizzes cədvəlinə insert tamamlandı.

📥 Enrollments insert olunur...
❌ Enrollments -> Sətir 0 insert Xətası: ('42000', '[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The incoming tabular data stream (TDS) remote procedure call (RPC) protocol stream is incorrect. Parameter 9 (""): The supplied value is not a valid instance of data type float. Check the source data for invalid values. An example of an invalid value is data of numeric type with scale greater than precision. (8023) (SQLExecDirectW)')
❌ Enrollments -> Sətir 2 insert Xətası: ('42000', '[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The incoming tabular data stream (TDS) remote procedure call (RPC) protocol stream is incorrect. Parameter 9 (""): The supplied value is not a valid instance of data type float. Check the source data for inval

In [ ]:
import pandas as pd
import pyodbc
import numpy as np

# SQL Server bağlantısı
conn = pyodbc.connect(
    r"DRIVER={ODBC Driver 17 for SQL Server};"
    r"SERVER=AZTRIP2\SQLEXPRESS;"
    r"DATABASE=OLP_Engagement_Analytics;"
    r"Trusted_Connection=yes;"
)
cursor = conn.cursor()


df = pd.read_csv("enrollments_2000.csv")


df = df.drop(columns=["enrollment_id"])

# NaN dəyərləri None ilə əvəz et
df = df.replace({np.nan: None})

# INSERT
for index, row in df.iterrows():
    cursor.execute("""
        INSERT INTO Enrollments (
            user_id, course_id, enrollment_date, completed,
            completion_date, final_grade, certificate_earned
        ) VALUES (?, ?, ?, ?, ?, ?, ?)
    """, 
    row.user_id,
    row.course_id,
    row.enrollment_date,
    row.completed,
    row.completion_date,
    row.final_grade,
    row.certificate_earned)

conn.commit()
cursor.close()
conn.close()

print("✅ Enrollments məlumatları uğurla insert edildi.")






✅ Enrollments məlumatları uğurla insert edildi.


In [9]:
import pandas as pd

df = pd.read_csv("enrollments_2000.csv")
print("CSV-dəki sütun adları bunlardır:")
print(df.columns.tolist())

CSV-dəki sütun adları bunlardır:
['enrollment_id', 'user_id', 'course_id', 'enrollment_date', 'completed', 'completion_date', 'final_grade', 'certificate_earned']


In [ ]:
import pandas as pd
import pyodbc
import numpy as np

# SQL Server bağlantısı
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=AZTRIP2\\SQLEXPRESS;"
    "DATABASE=OLP_Engagement_Analytics;"
    "Trusted_Connection=yes;"
)
cursor = conn.cursor()

# CSV faylını oxu
df = pd.read_csv("enrollments_2000.csv")

# Sütunları yoxla
expected_cols = ['user_id', 'course_id', 'enrollment_date', 'completed',
                 'completion_date', 'final_grade', 'certificate_earned']
df = df[expected_cols]

# final_grade sütununu float-a çevir və problemli dəyərləri None et
def convert_to_float(val):
    try:
        val = str(val).strip().replace('%', '')  
        if val == '' or val.lower() in ['nan', 'none', 'null']:
            return None
        return round(float(val), 2)
    except:
        return None

df['final_grade'] = df['final_grade'].apply(convert_to_float)

# Boş (NaT və NaN) dəyərləri None ilə əvəz etmək
df = df.replace({np.nan: None, 'nan': None})

# INSERT əməliyyatı
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO Enrollments (
            user_id, course_id, enrollment_date,
            completed, completion_date, final_grade, certificate_earned
        )
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """,
    row.user_id,
    row.course_id,
    row.enrollment_date,
    row.completed,
    row.completion_date,
    row.final_grade,
    row.certificate_earned)

# Dəyişiklikləri yadda saxlam 
conn.commit()
cursor.close()
conn.close()

print("✅ Bütün Enrollments dataları uğurla insert olundu")






✅ Bütün Enrollments dataları uğurla insert olundu və final_grade problemi tam aradan qaldırıldı.
